In [16]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

## The dataset

The dataset `data/chip_plants.txt` is a table of chip manufacturing plants taken from [Wikipedia](https://en.wikipedia.org/wiki/List_of_semiconductor_fabrication_plants) and is in the [Wiki Markup](https://en.wikipedia.org/wiki/Help:Introduction_to_editing_with_Wiki_Markup/2) format. In this file, each cell starts with the character `|`, like this:

~~~plain
|[[Texas Instruments]]
~~~

Square brackets  (`[[]]`) represent links to Wikipedia pages. In the example above, the code directs you to [this page](https://en.wikipedia.org/wiki/Texas_Instruments).

Each cell is on its own line, and table rows are demarcated by this code: `|-`. Thus,

~~~plain
|-
|[[Texas Instruments]] (formerly [[Semiconductor Manufacturing International Corporation|SMIC]] - Cension)
|Chengdu (CFAB)
| China {{flagicon|China}}, Chengdu
|
|
|200
|
|
|
|-
|[[Tsinghua Holdings|Tsinghua Unigroup]]<ref name="eetasia.com"/>
|
| China {{flagicon|China}}, Nanjing
|10 (first phase), 30
|Planned
|300
|
|100,000 (first phase)
|3D NAND Flash
|-
~~~

produces this:

![Table example](data/images/table_example.png)

Finally, `<ref name="example.com"/>` adds a footnote with a hyperlink (see the example in the second row above) and `{{flagicon|country_name}}` adds a country flag.

The file has two columns - `line` is the actual Wikipedia cells and `id` is an identifier of the chip manufacturing plant.

## Import Data

In [17]:
import pandas as pd

chips = pd.read_csv('C:/Users/Hevander/Desktop/DS4A/assignment1/chip_plants.csv')
chips.head(30)

,id,line
0,0,|[[United Microelectronics Corporation|UMC]] -...
1,0,|Fab 8N
2,0,| China {{flagicon|China}}
3,0,"|0.750,<ref name=""SEMI""/> 1.2, +0.5"
4,0,"|2003, May<ref name=""SEMI""/>"
5,0,|200
6,0,"|4000–1000, 500, 350, 250, 180, 110"
7,0,"|77,000"
8,0,|Foundry
9,0,|-


## Remove rows with extraneous information and blank spaces
---
[Series.str.strip()](https://docs.python.org/3/library/stdtypes.html#str.strip "Documentation") Return a copy of the string with the leading and trailing characters removed. The chars argument is a string specifying the set of characters to be removed. If omitted or None, the chars argument defaults to removing whitespace. 

In [18]:
def remove_separators(chips):
    """
    Removes all rows that consist of the separator `|-`
    
    Arguments:
    `chips`: A pandas DataFrame
    
    Outputs:
    `chips`: A pandas DataFrame (modified version of the input DataFrame)
    """
    
    # YOUR CODE HERE
    chips['line'] = chips['line'].str.strip()
    chips = chips[chips['line'] != '|-']
    
    return chips


In [19]:
# Call function here and set to a variable
stripped_chips = remove_separators(chips)
stripped_chips.head(20)

,id,line
0,0,|[[United Microelectronics Corporation|UMC]] -...
1,0,|Fab 8N
2,0,| China {{flagicon|China}}
3,0,"|0.750,<ref name=""SEMI""/> 1.2, +0.5"
4,0,"|2003, May<ref name=""SEMI""/>"
5,0,|200
6,0,"|4000–1000, 500, 350, 250, 180, 110"
7,0,"|77,000"
8,0,|Foundry
10,1,|[[United Microelectronics Corporation|UMC]]


[Series.str.replcace()](https://docs.python.org/3/library/stdtypes.html#str.replace "Documentation") Return a copy of the string with all occurrences of substring old replaced by new. If the optional argument count is given, only the first count occurrences are replaced.

[pandas.DataFrame.assign()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html "Documentation") Returns a new object with all original columns in addition to new ones. Existing columns that are re-assigned will be overwritten

In [20]:
def clean_cells(chips):
    """
    Removes unwanted characters from the `line` column after having
    removed the rows that consist of the separator `|-`
    
    Arguments:
    `chips`: A pandas DataFrame
    
    Outputs:
    `chips`: A pandas DataFrame (modified version of the input DataFrame)
    """
    chips = remove_separators(chips)
    
    # YOUR CODE HERE
    pd.set_option('mode.chained_assignment',None)
    remove_char = {'|','[',']'}
    for char in remove_char:
        chips['line'] = chips['line'].str.replace(char, '', regex=False)
    chips['line'] = chips['line'].str.strip()
    chips_line_clean = chips['line']
    chips_line_clean = chips_line_clean.str.replace(r"\<.*\>", "", regex=True)
    chips_line_clean = chips_line_clean.str.replace(r"\{.*\}", "", regex=True)
    chips = chips.assign(line=chips_line_clean)
    
    return chips


In [21]:
# Call function here and set to a variable
cleaned_chips = clean_cells(chips)
cleaned_chips.head(20)

,id,line
0,0,United Microelectronics CorporationUMC - Hejia...
1,0,Fab 8N
2,0,China
3,0,"0.750, 1.2, +0.5"
4,0,"2003, May"
5,0,200
6,0,"4000–1000, 500, 350, 250, 180, 110"
7,0,"77,000"
8,0,Foundry
10,1,United Microelectronics CorporationUMC


## Recreate the original Wikipedia table

Each table row in the Wikipedia dataset has to have exactly 9 columns:

1. `company`
2. `plant_name`
3. `plant_location`
4. `plant_cost_us_billions`
5. `started_production`
6. `wafer_size`
7. `process_technology`
8. `production_capacity`
9. `technology_products`

In [22]:
def assign_column_names(chips):
    """
    Assigns column names to the `chips` DataFrame after having
    cleaned the `list` column.
    
    Arguments:
    `chips`: A pandas DataFrame
    
    Outputs:
    `chips`: A pandas DataFrame (a modified version of the original input
    with the added column `columns`)
    """
    
    chips = clean_cells(chips)

    # YOUR CODE HERE
    col_names = ['company', 'plant_name', 'plant_location',
                'plant_cost_us_billions','started_production',
                'wafer_size', 'process_technology', 
                'production_capacity','technology_products']
    
    chips['columns'] = col_names*int((chips.shape[0]/len(col_names)))
    return chips


In [23]:
assign_column_names(chips)

,id,line,columns
0,0,United Microelectronics CorporationUMC - Hejia...,company
1,0,Fab 8N,plant_name
2,0,China,plant_location
3,0,"0.750, 1.2, +0.5",plant_cost_us_billions
4,0,"2003, May",started_production
...,...,...,...
5254,525,1959,started_production
5255,525,100/150,wafer_size
5256,525,900+,process_technology
5257,525,6000,production_capacity


[pandas.pivot()](https://pandas.pydata.org/docs/reference/api/pandas.pivot.html "Documentation") Reshape data (produce a “pivot” table) based on column values. Uses unique values from specified index / columns to form axes of the resulting DataFrame. 

In [24]:
# Use assign_column_names(df) with .pivot() to produce original table
assign_column_names(chips).pivot(index='id',columns="columns", values="line")

columns,company,plant_cost_us_billions,plant_location,plant_name,process_technology,production_capacity,started_production,technology_products,wafer_size
id,,,,,,,,,
0,United Microelectronics CorporationUMC - Hejia...,"0.750, 1.2, +0.5",China,Fab 8N,"4000–1000, 500, 350, 250, 180, 110","77,000","2003, May",Foundry,200
1,United Microelectronics CorporationUMC,0.35,"Taiwan , Hsinchu",Fab 6A,450,"31,000",1989,Foundry,150
2,United Microelectronics CorporationUMC,1,"Taiwan , Hsinchu",Fab 8AB,250,"67,000",1995,Foundry,200
3,United Microelectronics CorporationUMC,1,"Taiwan , Hsinchu",Fab 8C,350–110,"37,000",1998,Foundry,200
4,United Microelectronics CorporationUMC,1.5,"Taiwan , Hsinchu",Fab 8D,90,"31,000",2000,Foundry,200
...,...,...,...,...,...,...,...,...,...
521,Silex Microsystems,"0.009, 0.032","Sweden , Järfälla",,,,"2003, 2009",,
522,Integral,,"Belarus, Minsk",,"2000, 1.5 µm process1500, 350 nanometer350",,1963,,"100, 150, 200"
523,Crocus Nano Electronics,,"Russia , Moscow",CNE,65,4000,2015,"MRAM, RRAM, MEMS, Integrated passive devicesIP...",300


## Creating a function to output out csv file

In [25]:
def make_df_wide(chips):
    """
    Pivots the `chips` DataFrame and saves it as `clean_table.csv`,
    after having assigned a new column and having cleaned the data.
    
    Arguments:
    `chips`: A pandas DataFrame
    
    Outputs:
    No outputs.
    """
    chips = assign_column_names(chips)
    
    # YOUR CODE HERE
    chips = chips.pivot(index='id', columns="columns",values= 'line')
    chips.to_csv('C:/Users/Hevander/Desktop/DS4A/Data_indoor/BigThinkFiles/cleaned_table',index=False)

In [26]:
make_df_wide(chips)

### Checking for Missing Data

[glob.glob()](https://docs.python.org/3/library/glob.html "Documentation") Return a possibly-empty list of path names that match pathname, which must be a string containing a path specification.

[pandas.concat()](https://pandas.pydata.org/docs/reference/api/pandas.concat.html "Documentation") Concatenate pandas objects along a particular axis with optional set logic along the other axes.

In [27]:
import pandas as pd
import glob



def concat_csv(path):
    """
    ---Description here---
    
    Arguments:
    `input_variable`: data type
    
    Outputs:
    `output_variable`: data type and content discription
    """
    
    all_files = glob.glob(path + "/*.csv")
    li = []
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)
    frame = pd.concat(li, axis=0, ignore_index=True)
    return frame


In [32]:
# Set path string to a variable
path = 'C:/Users/Hevander/Desktop/DS4A/Data_indoor/BigThinkFiles'
# Call function and set to a variable
concat_df = concat_csv(path)
concat_df.head()

,UHF_42,Incident_Address_Borough,Date_Received,asbestos_count,dust_count,gasses_count,mold_count,sewageL_count,sewageO_count,ventilation_count
0,bayside - littleneck,Queens,2010,5.0,NaN,NaN,NaN,NaN,NaN,NaN
1,bayside - littleneck,Queens,2011,2.0,NaN,NaN,NaN,NaN,NaN,NaN
2,bayside - littleneck,Queens,2012,6.0,NaN,NaN,NaN,NaN,NaN,NaN
3,bayside - littleneck,Queens,2013,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,bayside - littleneck,Queens,2014,5.0,NaN,NaN,NaN,NaN,NaN,NaN


[DataFrame.isnull()](https://docs.python.org/3/library/glob.html "Documentation") Detect missing values.

[DataFrame.sum()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sum.html "Documentation") Return the sum of the values over the requested axis.

[DataFrame.sort_values()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sort_values.html "Documentation") Sort by the values along either axis.

In [29]:
def missing_data(frame):
    """
    ---Description here---
    
    Arguments:
    `input_variable`: data type
    
    Outputs:
    `output_variable`: data type and content discription
    """
    
    percent_missing = frame.isnull().sum() * 100 / len(frame)
    missing_value_df = pd.DataFrame({'column_name': frame.columns,
                                    'percent_missing': percent_missing})
    missing_value_df.sort_values('percent_missing', inplace=True)

    return missing_value_df

In [33]:
# Apply missing_data(df) to the concatinated dataframe
missing_data(concat_df)

,column_name,percent_missing
UHF_42,UHF_42,0.000000
Incident_Address_Borough,Incident_Address_Borough,0.000000
Date_Received,Date_Received,0.000000
gasses_count,gasses_count,82.178559
ventilation_count,ventilation_count,82.247501
asbestos_count,asbestos_count,82.488797
mold_count,mold_count,83.488452
dust_count,dust_count,83.626336
sewageO_count,sewageO_count,92.692175
sewageL_count,sewageL_count,93.278180


In [30]:
airQ_complaints = pd.read_csv('C:/Users/Hevander/Desktop/DS4A/Data_indoor/listing_complaints.csv')
airQ_complaints.head()

,Unnamed: 0,UHF_42,Incident_Address_Borough,Date_Received,asbestos_count,gasses_count,ventilation_count,mold_count,dust_count,sewageO_count,sewageL_count
0,0,bayside - littleneck,Queens,2010,5.0,4.0,4.0,3.0,NaN,NaN,NaN
1,1,bayside - littleneck,Queens,2011,2.0,8.0,8.0,2.0,NaN,NaN,NaN
2,2,bayside - littleneck,Queens,2012,6.0,16.0,3.0,2.0,NaN,NaN,NaN
3,3,bayside - littleneck,Queens,2013,1.0,12.0,4.0,NaN,NaN,NaN,NaN
4,4,bayside - littleneck,Queens,2014,5.0,21.0,NaN,NaN,4.0,NaN,NaN


In [31]:
airQ_missing = missing_data(airQ_complaints)
airQ_missing

,column_name,percent_missing
Unnamed: 0,Unnamed: 0,0.000000
UHF_42,UHF_42,0.000000
Incident_Address_Borough,Incident_Address_Borough,0.000000
Date_Received,Date_Received,0.000000
gasses_count,gasses_count,0.957854
ventilation_count,ventilation_count,1.340996
asbestos_count,asbestos_count,2.681992
mold_count,mold_count,8.237548
dust_count,dust_count,9.003831
sewageO_count,sewageO_count,59.386973
